## Import Statements

In [145]:
import gensim.downloader as api
import numpy as np
import re
import csv
import pandas as pd
import pprint
import string
import nltk
import sys
import spacy

from IPython.display import HTML
from nltk.corpus import wordnet 
from sklearn.manifold import TSNE
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize

pathToDatasets = '../datasets/'
pathToDataScripts = '../datasets/scripts/'
filePath = '../datasets/GoogleNews-vectors-negative300.bin'

sys.path.insert(0, pathToDataScripts)
from cleanDataset import tokenize_words 



from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

## Downloading binaries and models


In [146]:
tstString = input()
if("no" in tstString.lower()):
    print(" didnt reload model! ")
else:
    word_vectors = api.load("glove-wiki-gigaword-100")
    nltk.download('vader_lexicon')
    nltk.download('punkt')
    nltk.download('averaged_perceptron_tagger')
    nltk.download('wordnet')

no
 didnt reload model! 


## Global Variables and Global Objects

In [147]:
senty = SentimentIntensityAnalyzer()
vocabulary = word_vectors.vocab;

nlp = spacy.load('en')


NUMBER_OF_ALTERNATIVES = 5
TWEET_START = 50
NUM_OF_TWEETS = 50
VERBOSE_PRINTING = True
# VERBOSE_PRINTING = False

USE_SPACY = False
# USE_SPACY = True


punctuation = r"\"#$%&'+-/;<=>?@[\]*^_`{|}~"

## Class for Sentences


In [148]:
class SentenceWithHTML():
    
    def __init__(self,sentence,html):
        self.sentence = sentence;
        self.html = html;
    
    def getHTML(self):
        return self.html;
    
    def getSentence(self):
        return self.sentence
    
    def setHTML(self, html):
        self.html = html;
    
    def setSentence(self, sentence):
        self.sentence = sentence

class Sentence:
    
    
    def __init__(self, sentence, sentiment):
        self.ogSentence = sentence;
        self.ogSentiment = sentiment;
        self.indexToSetOfWords = {}
        self.alternateSentences = [];
        self.finalShiftSentences = [];

    def addAlternativesByIndex(self, index, listOfAlternatives):
        '''
            Adds the list of possible alternative words that 
            can be used per word based on the index of the word in the tokenized 
            sentence. (from cleanAndTokenizeText())
        '''
        if(self.indexToSetOfWords.get(index)):
            self.indexToSetOfWords[index] = self.indexToSetOfWords.union(set(listOfAlternatives))
        else:
            self.indexToSetOfWords[index] = set(listOfAlternatives)
        
    def addAlternativeStrings(self, strings):
        if(isinstance(strings,str)):
            self.alternateStrings = list(set(self.alternateStrings.append(strings)))
            self.alternateSentences.append(strings)
        else:
            self.alternateSentences.extend(strings)
    

    
    def addFinalSentences(self, sentences):
        if(isinstance(sentences, str)):
            self.finalShiftSentences.append(sentences)
        else:
            self.finalShiftSentences.extend(sentences)

    def resetFinalSentences(self):
        self.finalShiftSentences = [];
        
            

## Utility Code

In [149]:
def cstr(s, color='black'):
    return "<text style=color:{}>{}</text>".format(color, s)

def cleanAndTokenizeText(text):
    text = text.lower();
    newString = ""
    for char in text:
        if char not in punctuation:
            newString += char
    text = word_tokenize(newString)
    return text;

def getPOSTags(tweet):
    if(USE_SPACY == False):
        tags = nltk.pos_tag(tweet)
        return tags;    
    tweet = ' '.join(tweet)
    doc = nlp(tweet)
    tags = [(token.text, token.pos_) for token in doc] # since the format expected is [text,tag]
    return tags;
    

def getAntonymsOfWords(word):
    if(word not in vocabulary):
        return []
    setOfAntonyms = set()
    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            anton = l.antonyms()
            if(anton!=[]):
                setOfAntonyms.add(anton[0].name())
    if(len(setOfAntonyms) == 0):
        if(VERBOSE_PRINTING): print("No antonyms found for word {0}".format(word))
    return list(setOfAntonyms)

def listReplacements(word):
    if(word not in vocabulary):
        print(" --- {0} not in vocabulary ---".format(word))
        return []
    possibleReplacements = [word[0] for word in word_vectors.most_similar(word,topn=NUMBER_OF_ALTERNATIVES)]
    if(possibleReplacements == []):
        print(" --- No replacements for word {0} ---".format(word))
    antonyms = getAntonymsOfWords(word)
    if(antonyms != []):
        possibleReplacements.extend(antonyms)
        if(VERBOSE_PRINTING): print("Some antonyms for word {0} are {1}".format(word, antonyms[:3]))
        return possibleReplacements
    return possibleReplacements
    
def posApprovedReplacements(alternativeWords, userTokens, indexOfToken):
    if(alternativeWords == []):
        return []
    tempTokens = userTokens[:]
    POSTokens = getPOSTags(tempTokens)
    validWords = []
    
    mainTag = POSTokens[indexOfToken][1]
    mainWord = userTokens[indexOfToken]
    
    for ind,word in enumerate(alternativeWords):
        tempTokens[indexOfToken] = word
        posTags = getPOSTags(tempTokens)
        newTag = (posTags[indexOfToken])[1]
        
        if(str(newTag) == str(mainTag)):
            if(VERBOSE_PRINTING): print("Word {0}[{1}] replaced with {2}[{3}]".format(mainWord, mainTag, word,newTag))
            validWords.append(word)
    if(validWords == [] and VERBOSE_PRINTING):
        print("No POS words found for word {} with tag {}".format(mainWord, mainTag));
    return validWords
        
    
    
def getAlternativeSentences(sentenceObj):
    mainSentence = sentenceObj.ogSentence;
    mainSentiment = sentenceObj.ogSentiment;
    
    sentenceTokens = cleanAndTokenizeText(mainSentence)
   
    for ind, word in enumerate(sentenceTokens):
        alternativeSentenceWithHTML = []
        
        score = senty.polarity_scores(word)['compound']
        copyOfTokens = sentenceTokens[:]
        replacements = []
        if(score != 0.0):
            tempReplacements = listReplacements(word) # get embedding based relations
            if(tempReplacements == []):
                print("No replacements found at all for word {0}".format(word))
                continue
            replacements = posApprovedReplacements(tempReplacements[:], copyOfTokens[:], ind)
            if(replacements == []):
                print(" -- No POS approved words! -- for word {0}\n some non-POS:{1}".format(word, tempReplacements[:4]))
                continue
            sentenceObj.addAlternativesByIndex(ind, replacements)
            
            ## Generate new sentences by switching that word
            for newWord in replacements:
                htmlFriendlyTokens = copyOfTokens[:]
                copyOfTokens[ind] = newWord
                htmlFriendlyTokens[ind] = cstr(" <i>[{0}]</i>".format(newWord), "blue");
                newString = ' '.join(copyOfTokens)
                tSentence = SentenceWithHTML(newString, ' '.join(htmlFriendlyTokens))
                alternativeSentenceWithHTML.append(tSentence)
        sentenceObj.addAlternativeStrings(alternativeSentenceWithHTML)
    return sentenceObj

def shiftSentiment(sentenceObj, positive=True):
    
    actualTweet = sentenceObj.ogSentence;
    mainSentiment = sentenceObj.ogSentiment;
    alternateTweets = sentenceObj.alternateSentences;
    
    
    happiestTweet = ""
    saddestTweet = ""
    happiestScore = -sys.maxsize - 1
    saddestScore = sys.maxsize
    correctTweets = []
    
    for tSentence in alternateTweets:
        tweet = tSentence.getSentence();
        newSenty = senty.polarity_scores(tweet)['compound']
        
        if(newSenty < saddestScore):
            saddestTweet = tSentence
            saddestScore = newSenty
        if(newSenty > happiestScore):
            happiestTweet = tSentence
            happiestScore = newSenty
            
        if(newSenty == mainSentiment):
            continue
        if(positive == True):
            if(newSenty > mainSentiment):
                correctTweets.append(tSentence)
                continue
            
            elif(newSenty < mainSentiment):
                continue
                # Grab happiest tweet and if it's not "", then generate more happy tweets from it
        if(positive == False):
            if(newSenty < mainSentiment):
                correctTweets.append(tSentence)
                continue
            
            elif(newSenty > mainSentiment):
                continue
                # grab happiest tweet, and if it's not "", then generate more happy tweets from it
                
    if(correctTweets == []):
        print("\n\nNo tweets found when trying to do Positive={}\n\n".format(positive))
    sentenceObj.resetFinalSentences();
    sentenceObj.addFinalSentences(correctTweets);
    return sentenceObj;

In [150]:
def printStrings(sentenceObj):
    newStrings = sentenceObj.finalShiftSentences;
    mainSentiment = sentenceObj.ogSentiment;    
    for ind, tSentence in enumerate(newStrings):
            alteredTweet = tSentence.getSentence()
            htmlText = tSentence.getHTML()
            sentimentOfNewString = senty.polarity_scores(alteredTweet)['compound']
            if(sentimentOfNewString == mainSentiment or sentimentOfNewString == 0):
                display(HTML(cstr("{0}: {1}".format(htmlText,sentimentOfNewString),'DarkGray')))
            elif(sentimentOfNewString > mainSentiment):
                display(HTML(cstr("{0}: {1}".format(htmlText,sentimentOfNewString),'green')))
            else:
                display(HTML(cstr("{0}: {1}".format(htmlText,sentimentOfNewString),'red')))

In [151]:
def runThroughTweets():
    
    tweets_df = pd.read_csv( pathToDatasets + 'cleanedTweets.csv' , nrows=NUM_OF_TWEETS, skiprows=TWEET_START)

    tweets = tweets_df.values

    listOfObjects = []
    for counter,tweet in enumerate(tweets):
        tweet = tweet[0]
        mainSentiment = senty.polarity_scores(tweet)['compound']
        if(mainSentiment == 0):
            continue
        print("\n {0}:{1}\n".format(tweet,mainSentiment))   
        sentenceObj = Sentence(tweet, mainSentiment)
        sentenceObj = getAlternativeSentences(sentenceObj)
        alternateTweets = (sentenceObj.alternateSentences)[:]
        if(alternateTweets == [] or alternateTweets == None):
            print(" -- No new Strings generated ---\n\n")
            continue
        sentenceObj = shiftSentiment(sentenceObj, True);
        printStrings(sentenceObj)
        sentenceObj = shiftSentiment(sentenceObj, False);
        printStrings(sentenceObj)
    
runThroughTweets()


 broadband plan a massive broken promise  via  still waiting for broadband we are :-0.2023

Some antonyms for word broken are ['promote', 'unbroken', 'repair']
Word broken[NN] replaced with breaking[NN]
Word broken[NN] replaced with broke[NN]
Word broken[NN] replaced with apart[NN]
Word broken[NN] replaced with promote[NN]
Word broken[NN] replaced with repair[NN]
Word broken[NN] replaced with conform_to[NN]
Word broken[NN] replaced with keep[NN]
Word broken[NN] replaced with make[NN]
No antonyms found for word promise
Word promise[NN] replaced with pledge[NN]
Word promise[NN] replaced with commitment[NN]
Word promise[NN] replaced with hope[NN]



  wow tons of replies from you may have to unfollow so i can see my friends tweets you are scrolling the feed a lot. :0.7845

No antonyms found for word wow
Word wow[JJ] replaced with c'mon[JJ]
Word wow[JJ] replaced with hey[JJ]
Some antonyms for word friends are ['foe', 'stranger']
Word friends[NNS] replaced with parents[NNS]
Word friends[NNS] replaced with relatives[NNS]


No tweets found when trying to do Positive=True





 put vacation photos online a few yrs ago. pc crashed and now i forget the name of the site. :-0.2263

Some antonyms for word forget are ['mind', 'remember']
Word forget[VBP] replaced with remember[VBP]
Word forget[VBP] replaced with tell[VBP]
Word forget[VBP] replaced with imagine[VBP]
Word forget[VBP] replaced with mind[VBP]
Word forget[VBP] replaced with remember[VBP]




No tweets found when trying to do Positive=False



 i need a hug :0.4767

No antonyms found for word hug
Word hug[NN] replaced with hugs[NN]
Word hug[NN] replaced with kiss[NN]
Word hug[NN] replaced with hugged[NN]
Word hug[NN] replaced with goodbye[NN]



  not sure what they are only that they are pos! as much as i want to i dont think can trade away company assets sorry andy! :-0.2134

Some antonyms for word sure are ['uncertain', 'unsure']
Word sure[JJ] replaced with uncertain[JJ]
Word sure[JJ] replaced with unsure[JJ]
No antonyms found for word want
Word want[VBP] replaced with do[VBP]
Word want[VBP] replaced with know[VBP]
Word want[VBP] replaced with get[VBP]
Some antonyms for word assets are ['liability']
Word assets[NNS] replaced with investments[NNS]
Word assets[NNS] replaced with funds[NNS]
Some antonyms for word sorry are ['unregretful']
Word sorry[VBP] replaced with 'm[VBP]
Word sorry[VBP] replaced with glad[VBP]


...


  hmm   do u really enjoy being with him  if the problems are too constants u should think things more  find someone ulike:0.2006

Some antonyms for word enjoy are ['suffer']
Word enjoy[VB] replaced with prefer[VB]
Word enjoy[VB] replaced with feel[VB]
Word enjoy[VB] replaced with suffer[VB]
No antonyms found for word problems
Word problems[NNS] replaced with difficulties[NNS]
Word problems[NNS] replaced with troubles[NNS]



 strider is a sick little puppy  :-0.5106

Some antonyms for word sick are ['well', 'keep_down']
Word sick[JJ] replaced with ill[JJ]
Word sick[JJ] replaced with pregnant[JJ]
Word sick[JJ] replaced with elderly[JJ]
Word sick[JJ] replaced with keep_down[JJ]




No tweets found when trying to do Positive=False



 so ryleegrace...wana go steve is party or not sadly since its easter i wnt b able 2 do much  but ohh well.....:0.3645

No antonyms found for word party
Word party[NN] replaced with opposition[NN]
Word party[NN] replaced with coalition[NN]
Word party[NN] replaced with leader[NN]
Some antonyms for word sadly are ['happily']
Word sadly[RB] replaced with thankfully[RB]
Word sadly[RB] replaced with terribly[RB]
Word sadly[RB] replaced with frankly[RB]
Word sadly[RB] replaced with curiously[RB]
Word sadly[RB] replaced with painfully[RB]
Word sadly[RB] replaced with happily[RB]
Some antonyms for word well are ['badly', 'ill', 'disadvantageously']
Word well[RB] replaced with so[RB]
Word well[RB] replaced with badly[RB]
Word well[RB] replaced with disadvantageously[RB]



 hey i actually won one of my bracket pools! too bad it was not the one for money :0.126

Some antonyms for word won are ['fail', 'fall_back', 'lost']
No POS words found for word won with tag VBD
 -- No POS approved words! -- for word won
 some non-POS:['winning', 'win', 'wins', 'winner']
Some antonyms for word bad are ['goodness', 'good', 'unregretful']
Word bad[JJ] replaced with good[JJ]
Word bad[JJ] replaced with goodness[JJ]
Word bad[JJ] replaced with good[JJ]
Word bad[JJ] replaced with unregretful[JJ]




No tweets found when trying to do Positive=False



 a bad nite for the favorite teams astros and spartans lose.  the nite out with t.w. was good.:-0.0772

Some antonyms for word bad are ['goodness', 'good', 'unregretful']
Word bad[JJ] replaced with good[JJ]
Word bad[JJ] replaced with good[JJ]
Word bad[JJ] replaced with unregretful[JJ]
No antonyms found for word favorite
Word favorite[JJ] replaced with favourite[JJ]
Word favorite[JJ] replaced with popular[JJ]
Word favorite[JJ] replaced with famous[JJ]
Some antonyms for word lose are ['profit', 'keep', 'find']
Word lose[VBP] replaced with get[VBP]
Word lose[VBP] replaced with keep[VBP]
Word lose[VBP] replaced with find[VBP]
Word lose[VBP] replaced with gain[VBP]
Word lose[VBP] replaced with win[VBP]
Some antonyms for word good are ['evilness', 'bad', 'evil']
Word good[JJ] replaced with sure[JJ]
Word good[JJ] replaced with bad[JJ]
Word good[JJ] replaced with evil[JJ]
Word good[JJ] replaced with ill[JJ]


## Testing Code


In [152]:
specificWord = "good"
def testOneWord(word=""):
    if(word==""):
        return
    print(word)

## Main Cell


In [153]:
specificString = ""
def specificString(textString=""):
    if(textString == "" or textString == None):
        return
    mainSentiment = senty.polarity_scores(textString)['compound']
    if(mainSentiment == 0):
        print("{} \n No sentiment found in sentence".format(textString));
        return;
    print("\n {0}:{1}\n".format(textString,mainSentiment))   
    sentenceObj = Sentence(textString, mainSentiment)
    sentenceObj = getAlternativeSentences(sentenceObj)
    alternateTweets = (sentenceObj.alternateSentences)[:]
    if(alternateTweets == [] or alternateTweets == None):
        print(" -- No new Strings generated ---\n\n")
        return
    sentenceObj = shiftSentiment(sentenceObj, True);
    printStrings(sentenceObj)
    sentenceObj = shiftSentiment(sentenceObj, False);
    printStrings(sentenceObj)
    
# specificString("Bristol University has a wonderful worklife balance")
# specificString("Bristol University has an incredibly good worklife balance")
# specificString("Bristol University has a good worklife balance")
# specificString("Bristol University has an incredible worklife balance")
specificString("This was hard but fun to make!")


 This was hard but fun to make!:0.6749

Some antonyms for word hard are ['voiced', 'soft', 'easy']
Word hard[JJ] replaced with tough[JJ]
Word hard[JJ] replaced with difficult[JJ]
Word hard[JJ] replaced with soft[JJ]
Word hard[JJ] replaced with easy[JJ]
No antonyms found for word fun
No POS words found for word fun with tag NN
 -- No POS approved words! -- for word fun
 some non-POS:['stuff', 'wonderful', 'entertaining', 'really']
